add database to google api

- Create Agents
- Define tools
- Create graph
- Define Agent Nodes
- Define Tool Node
- Define Edge Logic
- Define the Graph
- show the graph
- Invoke
  
- Human in the loop
- Maximum number of steps
- Helper Utilities

# LLM Agents for Clinical Trial Management

In this notebook, we develop an agent-based system to automate the management and analysis of clinical trial data, ensuring compliance with regulatory standards, and providing comprehensive reports for stakeholders. This pipeline also incorporate human-in-the-loop (HITL) and memory mechanisms to enhance functionality and accuracy.

## Steps and Plan

1. Define the Problem:

Objective: Automate clinical trial data management, analysis, compliance checks, and reporting.

Key Components: Data collection, data analysis, compliance verification, reporting, and human-in-the-loop interventions.

2. Design the Architecture:
**Agents:**
Data Collection Agent: Gathers patient data, trial results, and other relevant information.

Data Analysis Agent: Analyzes the collected data to generate reports on trial outcomes.

Compliance Agent: Ensures all data and processes adhere to regulatory standards.

Reporting Agent: Compiles and presents the analysis findings in comprehensive reports.

Human-in-the-Loop: Integrate a review phase where human experts validate data analysis and compliance checks before final reporting.

Memory: Utilize LangGraph’s memory capabilities to track the progress and history of each trial.

The Clinical Trial Matching Agent is designed to match patients wit	h suitable clinical trials based on their medical history, symptoms, and ongoing treatments. This application helps patients find relevant clinical trials they may be eligible for, improving their chances of accessing new treatments and contributing to medical research.


1. Database Setup
- SQLite Database: Create two databases:
- Patients DB: Stores patient medical history, previous trials, and other relevant data.
- Clinical Trials DB: Contains information about ongoing and upcoming clinical trials, including eligibility criteria.
2. Agents and Tools
- Patient Data Ingestion Agent: Retrieves patient data from the SQLite database or prompts the user to provide missing data.
- Data Validation and Collection Loop: Ensures all required fields are collected using persistence.
- Policy Compliance Agent: Cross-checks patient data with institutional policies.
- Trial Vectorization Agent: Extracts and vectorizes recruiting trials for efficient search.
- Trial Matching Agent: Forms prompts from patient data to search the vector database for potential trial matches.
- Human-in-the-Loop (HIL) Agent: Presents top trial matches to the user for selection.
- Persistence Mechanism: Allows the application to save and resume states, especially useful for collecting patient data and searching trials.

In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
# _set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Clinical Trial Management"

In [2]:
# import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

import pprint
# A function for printing nicely
def nprint(text, indent=2):
    pp = pprint.PrettyPrinter(indent=indent)
    pp.pprint(text)

In [25]:
import pandas as pd
from datetime import datetime, timedelta

# Generate the sample data again for display
columns = ["patient_id", "name", "age", "medical_history", "previous_trials", "trial_status", "last_trial_dates"]
data = []

import random

# Given names and surnames
names = ["John", "Jane", "Alice", "Michael", "Emily", "Daniel", "Sophia", "James", "Emma", "Oliver"]
surnames = ["Doe", "Smith", "Johnson", "Brown", "Davis", "Garcia", "Martinez", "Anderson", "Thomas", "Wilson"]

# Generate all possible unique combinations of names and surnames
combinations = [(name, surname) for name in names for surname in surnames]

# Shuffle the combinations to ensure randomness
random.shuffle(combinations)

# Select the first 100 unique combinations
unique_names = combinations[:100]

# Generate the full names
full_names = [f"{name} {surname}" for name, surname in unique_names]

# Display the first few full names for verification
print(full_names[:10])

# Optionally, you can also display all 100 unique full names
# print(len(full_names))
# unique entries of list full_names
print(len(set(full_names)))

medical_conditions = ["Hypertension", "Diabetes", "Asthma", "Heart Disease", "Arthritis",
                      "Chronic Pain", "Anxiety", "Depression", "Cancer", "Obesity"]
trial_statuses = ["Completed", "Ongoing", "Withdrawn"]

def random_date(start, end):
    return start + timedelta(days=random.randint(0, int((end - start).days)))

# start_date must be 2 years before now
start_date = datetime.now() - timedelta(days=365 * 2)
# start_date = datetime(2020, 1, 1)

# end_date must be a month before now
end_date = datetime.now() - timedelta(days=10)
# end_date = datetime(2023, 1, 1)

for i in range(1, 101):
    name = random.choice(full_names)
    age = random.randint(20, 80)
    medical_history = random.choice(medical_conditions)
    
    if random.choice([True, False]):
        previous_trials = f"Trial {random.randint(1, 20)}"
        trial_status = random.choice(trial_statuses)
        last_trial_dates = random_date(start_date, end_date).strftime('%Y-%m-%d')
    else:
        previous_trials = ""
        trial_status = ""
        last_trial_dates = ""

    data.append((i, name, age, medical_history, previous_trials, trial_status, last_trial_dates))

df = pd.DataFrame(data, columns=columns)
# save df to csv
df.to_csv("patients.csv", index=False)
df.head(20)

['Jane Thomas', 'Emma Doe', 'John Wilson', 'Emily Johnson', 'Sophia Thomas', 'James Garcia', 'Alice Wilson', 'Alice Smith', 'Michael Smith', 'Daniel Doe']
100


,patient_id,name,age,medical_history,previous_trials,trial_status,last_trial_dates
0,1,Jane Davis,36,Asthma,Trial 7,Ongoing,2024-02-29
1,2,Daniel Davis,53,Heart Disease,,,
2,3,Jane Doe,77,Diabetes,Trial 11,Ongoing,2022-12-20
3,4,Emily Smith,43,Cancer,,,
4,5,Alice Johnson,42,Anxiety,Trial 17,Ongoing,2024-05-08
5,6,Michael Smith,50,Obesity,,,
6,7,Oliver Brown,71,Chronic Pain,Trial 3,Withdrawn,2022-12-08
7,8,Oliver Davis,68,Obesity,Trial 10,Completed,2023-06-26
8,9,Oliver Davis,69,Obesity,,,
9,10,Emma Anderson,67,Anxiety,Trial 11,Completed,2022-10-16


In [26]:
import os
# import shutil
import sqlite3

import pandas as pd

overwrite = False

database_file = 'patients_database.db'
df = pd.read_csv('patients.csv')
if overwrite or not os.path.exists(database_file):
    if os.path.exists(database_file):
        os.remove(database_file)
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS patients (
        patient_id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        medical_history TEXT,
        previous_trials TEXT,
        trial_status TEXT,
        last_trial_dates TEXT
    )
    ''')

    # Insert DataFrame into SQLite table
    df.to_sql('patients', conn, if_exists='append', index=False)

    # Commit and close the connection
    conn.commit()
else:
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()

query = 'SELECT * FROM patients'

# Execute the query and fetch all results
cursor.execute(query)
rows = cursor.fetchall()

# Optionally, you can get the column names
column_names = [description[0] for description in cursor.description]

# Convert the results to a Pandas DataFrame for better readability
df = pd.DataFrame(rows, columns=column_names)

# Display the DataFrame
conn.close()

# db = database_file
df.head(10)

,patient_id,name,age,medical_history,previous_trials,trial_status,last_trial_dates
0,1,Jane Martinez,68,Chronic Pain,Trial 4,Withdrawn,2020-05-22
1,2,Sophia Thomas,76,Chronic Pain,Trial 18,Completed,2022-06-18
2,3,Oliver Anderson,51,Hypertension,Trial 19,Completed,2020-01-23
3,4,Emily Martinez,51,Depression,None,None,None
4,5,Alice Garcia,70,Chronic Pain,Trial 7,Completed,2021-07-24
5,6,John Anderson,26,Anxiety,Trial 15,Withdrawn,2021-06-30
6,7,Michael Davis,73,Anxiety,None,None,None
7,8,Alice Wilson,64,Cancer,Trial 17,Ongoing,2022-09-25
8,9,Emma Wilson,34,Heart Disease,None,None,None
9,10,Emma Doe,44,Arthritis,None,None,None


## Dummy tests

In [12]:
assert 1==2

     type                  name              tbl_name  rootpage  \
0   table        aircrafts_data        aircrafts_data         2   
1   table         airports_data         airports_data         3   
2   table       boarding_passes       boarding_passes         4   
3   table              bookings              bookings         5   
4   table               flights               flights         6   
5   table                 seats                 seats         7   
6   table        ticket_flights        ticket_flights         8   
7   table               tickets               tickets         9   
8   table           car_rentals           car_rentals     21958   
9   table                hotels                hotels     21961   
10  table  trip_recommendations  trip_recommendations     21962   

                                                  sql  
0   CREATE TABLE "aircrafts_data" (\n"aircraft_cod...  
1   CREATE TABLE "airports_data" (\n"airport_code"...  
2   CREATE TABLE "boarding_

In [ ]:

import os
import shutil
import sqlite3

import pandas as pd
import requests

db_url = "https://storage.googleapis.com/benchmarks-artifacts/travel-db/travel2.sqlite"
local_file = "travel2.sqlite"
# The backup lets us restart for each tutorial section
backup_file = "travel2.backup.sqlite"
overwrite = False
if overwrite or not os.path.exists(local_file):
    response = requests.get(db_url)
    response.raise_for_status()  # Ensure the request was successful
    with open(local_file, "wb") as f:
        f.write(response.content)
    # Backup - we will use this to "reset" our DB in each section
    shutil.copy(local_file, backup_file)
# Convert the flights to present time for our tutorial
# local_file = 
conn = sqlite3.connect(local_file)
cursor = conn.cursor()

query = 'SELECT * FROM sqlite_master'

# Execute the query and fetch all results
cursor.execute(query)
rows = cursor.fetchall()

# Optionally, you can get the column names
column_names = [description[0] for description in cursor.description]

# Convert the results to a Pandas DataFrame for better readability
df = pd.DataFrame(rows, columns=column_names)

# Display the DataFrame
print(df)

In [15]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn).name.tolist()
tables
tdf = {}
for t in tables:
    tdf[t] = pd.read_sql(f"SELECT * from {t}", conn)


In [21]:
tdf['flights'] 
example_time = pd.to_datetime(tdf["flights"]["actual_departure"].replace("\\N", pd.NaT)).max()
current_time = pd.to_datetime("now").tz_localize(example_time.tz)
time_diff = current_time - example_time
time_diff

Timedelta('50 days 08:36:51.523473')

In [22]:
tdf["bookings"]["book_date"]

0         2024-03-20 01:21:03.561731+00:00
1         2024-03-29 07:11:03.561731+00:00
2         2024-04-30 12:36:03.561731+00:00
3         2024-04-25 11:37:03.561731+00:00
4         2024-04-22 19:49:03.561731+00:00
                        ...               
262783    2024-04-01 05:32:03.561731+00:00
262784    2024-04-23 04:04:03.561731+00:00
262785    2024-04-04 18:51:03.561731+00:00
262786    2024-04-23 02:54:03.561731+00:00
262787    2024-03-16 20:21:03.561731+00:00
Name: book_date, Length: 262788, dtype: object

In [ ]:

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
).name.tolist()
tdf = {}
for t in tables:
    tdf[t] = pd.read_sql(f"SELECT * from {t}", conn)

example_time = pd.to_datetime(
    tdf["flights"]["actual_departure"].replace("\\N", pd.NaT)
).max()
current_time = pd.to_datetime("now").tz_localize(example_time.tz)
time_diff = current_time - example_time

tdf["bookings"]["book_date"] = (
    pd.to_datetime(tdf["bookings"]["book_date"].replace("\\N", pd.NaT), utc=True)
    + time_diff
)

datetime_columns = [
    "scheduled_departure",
    "scheduled_arrival",
    "actual_departure",
    "actual_arrival",
]
for column in datetime_columns:
    tdf["flights"][column] = (
        pd.to_datetime(tdf["flights"][column].replace("\\N", pd.NaT)) + time_diff
    )

for table_name, df in tdf.items():
    df.to_sql(table_name, conn, if_exists="replace", index=False)
del df
del tdf
conn.commit()
conn.close()

db = local_file  # We'll be using this local file as our DB in this tutorial

In [6]:
tdf

NameError: name 'tdf' is not defined